[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain-academy/blob/main/module-0/basics.ipynb) [![Open in LangChain Academy](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e9eba12c7b7688aa3dbb5e_LCA-badge-green.svg)](https://academy.langchain.com/courses/take/intro-to-langgraph/lessons/56295530-getting-set-up-video-guide)

# LangChain Academy

Welcome to LangChain Academy! 

## Context

At LangChain, we aim to make it easy to build LLM applications. One type of LLM application you can build is an agent. There’s a lot of excitement around building agents because they can automate a wide range of tasks that were previously impossible. 

In practice though, it is incredibly difficult to build systems that reliably execute on these tasks. As we’ve worked with our users to put agents into production, we’ve learned that more control is often necessary. You might need an agent to always call a specific tool first or use different prompts based on its state. 

To tackle this problem, we’ve built [LangGraph](https://langchain-ai.github.io/langgraph/) — a framework for building agent and multi-agent applications. Separate from the LangChain package, LangGraph’s core design philosophy is to help developers add better precision and control into agent workflows, suitable for the complexity of real-world systems.

## Course Structure

The course is structured as a set of modules, with each module focused on a particular theme related to LangGraph. You will see a folder for each module, which contains a series of notebooks. A video will accompany each notebook to help walk through the concepts, but the notebooks are also stand-alone, meaning that they contain explanations and can be viewed independently of the videos. Each module folder also contains a `studio` folder, which contains a set of graphs that can be loaded into [LangGraph Studio](https://github.com/langchain-ai/langgraph-studio), our IDE for building LangGraph applications.

## Setup

Before you begin, please follow the instructions in the `README` to create an environment and install dependencies.

## Chat models

In this course, we'll be using [Chat Models](https://python.langchain.com/v0.2/docs/concepts/#chat-models), which do a few things take a sequence of messages as inputs and return chat messages as outputs. LangChain does not host any Chat Models, rather we rely on third party integrations. [Here](https://python.langchain.com/v0.2/docs/integrations/chat/) is a list of 3rd party chat model integrations within LangChain! By default, the course will use [ChatOpenAI](https://python.langchain.com/v0.2/docs/integrations/chat/openai/) because it is both popular and performant. As noted, please ensure that you have an `OPENAI_API_KEY`.

Let's check that your `OPENAI_API_KEY` is set and, if not, you will be asked to enter it.

In [9]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community langchain-tavily

Traceback (most recent call last):
  File "/Users/haozhezhang/Documents/Agents/News_agent/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3579, in run_code
  File "/var/folders/rd/wvk7c0712r5czjzzfb1mh5b40000gn/T/ipykernel_5858/2766978719.py", line 1, in <module>
    get_ipython().run_line_magic('pip', 'install --quiet -U langchain_openai langchain_core langchain_community langchain-tavily')
  File "/Users/haozhezhang/Documents/Agents/News_agent/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 2482, in run_line_magic
  File "/Users/haozhezhang/Documents/Agents/News_agent/.venv/lib/python3.10/site-packages/IPython/core/magics/packaging.py", line 105, in pip
  File "/Users/haozhezhang/Documents/Agents/News_agent/.venv/lib/python3.10/site-packages/ipykernel/zmqshell.py", line 740, in system_piped
  File "/Users/haozhezhang/Documents/Agents/News_agent/.venv/lib/python3.10/site-packages/IPython/utils/_process_posix.py", line 125, in syste

In [2]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("ZHIPUAI_API_KEY")

[Here](https://python.langchain.com/v0.2/docs/how_to/#chat-models) is a useful how-to for all the things that you can do with chat models, but we'll show a few highlights below. If you've run `pip install -r requirements.txt` as noted in the README, then you've installed the `langchain-openai` package. With this, we can instantiate our `ChatOpenAI` model object. If you are signing up for the API for the first time, you should receive [free credits](https://community.openai.com/t/understanding-api-limits-and-free-tier/498517) that can be applied to any of the models. You can see pricing for various models [here](https://openai.com/api/pricing/). The notebooks will default to `gpt-4o` because it's a good balance of quality, price, and speed [see more here](https://help.openai.com/en/articles/7102672-how-can-i-access-gpt-4-gpt-4-turbo-gpt-4o-and-gpt-4o-mini), but you can also opt for the lower priced `gpt-3.5` series models. 

There are [a few standard parameters](https://python.langchain.com/v0.2/docs/concepts/#chat-models) that we can set with chat models. Two of the most common are:

* `model`: the name of the model
* `temperature`: the sampling temperature

`Temperature` controls the randomness or creativity of the model's output where low temperature (close to 0) is more deterministic and focused outputs. This is good for tasks requiring accuracy or factual responses. High temperature (close to 1) is good for creative tasks or generating varied responses. 

In [3]:
from langchain_openai import ChatOpenAI
glm4p5_chat = ChatOpenAI(
    model="glm-4.5-flash",
    temperature=0,
    api_key=os.environ["ZHIPUAI_API_KEY"],
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
    )
glm4_chat = ChatOpenAI(
    model="glm-4-flash",
    temperature=0,
    api_key=os.environ["ZHIPUAI_API_KEY"],
    openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
    )

Chat models in LangChain have a number of [default methods](https://python.langchain.com/v0.2/docs/concepts/#runnable-interface). For the most part, we'll be using:

* `stream`: stream back chunks of the response
* `invoke`: call the chain on an input

And, as mentioned, chat models take [messages](https://python.langchain.com/v0.2/docs/concepts/#messages) as input. Messages have a role (that describes who is saying the message) and a content property. We'll be talking a lot more about this later, but here let's just show the basics.

In [4]:
from langchain_core.messages import HumanMessage

# Create a message
msg = HumanMessage(content="Hello world", name="Lance")

# Message list
messages = [msg]

# Invoke the model with a list of messages 
glm4p5_chat.invoke(messages)

AIMessage(content='Here\'s a simple "Hello, World!" program in Python:\n\n```python\nprint("Hello, World!")\n```\n\n### Explanation:\n1. **`print()`**: This is a built-in Python function that outputs text to the console.\n2. **`"Hello, World!"`**: This is a string (text) enclosed in double quotes. The content inside the quotes is displayed exactly as written.\n\n### How to Run:\n1. Save the code in a file named `hello.py`.\n2. Open a terminal/command prompt.\n3. Navigate to the directory where you saved the file.\n4. Run the command:  \n   ```bash\n   python hello.py\n   ```\n5. You\'ll see the output:  \n   ```\n   Hello, World!\n   ```\n\n### Key Notes:\n- Python is case-sensitive, so `print` must be lowercase.\n- The quotes can be single (`\'`) or double (`"`), but they must match.\n- This program works in all Python versions (2.x and 3.x), though Python 3.x is recommended.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 302, 'prompt_to

We get an `AIMessage` response. Also, note that we can just invoke a chat model with a string. When a string is passed in as input, it is converted to a `HumanMessage` and then passed to the underlying model.


In [ ]:
glm4p5_chat.invoke("hello world")

AIMessage(content='Here\'s a simple "Hello, World!" program in Python:\n\n```python\nprint("Hello, World!")\n```\n\n### Explanation:\n1. `print()` is a built-in Python function that outputs text to the console.\n2. `"Hello, World!"` is a string literal (text enclosed in quotes) that gets printed.\n\n### How to run it:\n1. Save the code in a file named `hello.py`\n2. Run it using Python:\n   ```bash\n   python hello.py\n   ```\n\n### Output:\n```\nHello, World!\n```\n\n### Alternative versions in other languages:\n\n**JavaScript:**\n```javascript\nconsole.log("Hello, World!");\n```\n\n**Java:**\n```java\npublic class HelloWorld {\n    public static void main(String[] args) {\n        System.out.println("Hello, World!");\n    }\n}\n```\n\n**C++:**\n```cpp\n#include <iostream>\nint main() {\n    std::cout << "Hello, World!" << std::endl;\n    return 0;\n}\n```\n\n**C#:**\n```csharp\nusing System;\nclass Program {\n    static void Main() {\n        Console.WriteLine("Hello, World!");\n    

: 

: 

In [ ]:
glm4_chat.invoke("hello world")

AIMessage(content="Hello 👋! I'm ChatGLM, the artificial intelligence assistant, nice to meet you. How can I help you today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 7, 'total_tokens': 36, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'glm-4-flash', 'system_fingerprint': None, 'id': '20251021133842730225fa2c454546', 'service_tier': None, 'finish_reason': 'stop', 'logprobs': None}, id='run--3fefadc2-60cb-4707-9ee4-b77c66efbb82-0', usage_metadata={'input_tokens': 7, 'output_tokens': 29, 'total_tokens': 36, 'input_token_details': {}, 'output_token_details': {}})

: 

: 

The interface is consistent across all chat models and models are typically initialized once at the start up each notebooks. 

So, you can easily switch between models without changing the downstream code if you have strong preference for another provider.


## Search Tools

You'll also see [Tavily](https://tavily.com/) in the README, which is a search engine optimized for LLMs and RAG, aimed at efficient, quick, and persistent search results. As mentioned, it's easy to sign up and offers a generous free tier. Some lessons (in Module 4) will use Tavily by default but, of course, other search tools can be used if you want to modify the code for yourself.

In [ ]:
_set_env("TAVILY_API_KEY")

: 

: 

In [ ]:
from langchain_tavily import TavilySearch  # updated at 1.0

tavily_search = TavilySearch(max_results=3)

data = tavily_search.invoke({"query": "AAPL NEWS?"})
search_docs = data.get("results", data)

: 

: 

In [ ]:
search_docs

[{'url': 'https://www.cnn.com/markets/stocks/AAPL',
  'title': 'AAPL Stock Quote Price and Forecast - CNN',
  'content': '[Markets](https://www.cnn.com/markets) [More](https://www.cnn.com/markets/stocks/AAPL) *   [Sign out](https://www.cnn.com/markets/stocks/AAPL#) *   [Sign out](https://www.cnn.com/markets/stocks/AAPL#) [Quote](https://www.cnn.com/markets/stocks/AAPL#quote "Quote") [About](https://www.cnn.com/markets/stocks/AAPL#about "About") [News](https://www.cnn.com/markets/stocks/AAPL#news "News") [Financials](https://www.cnn.com/markets/stocks/AAPL#financials "Financials") [Competitors](https://www.cnn.com/markets/stocks/AAPL#competitors "Competitors") [Yesterday 6:42am ET Apple (AAPL) Sees Strongest iPhone Growth Since the Pandemic by TipRanks](https://www.tipranks.com/news/apple-aapl-sees-strongest-iphone-growth-since-the-pandemic?utm_source=edition.cnn.com&utm_medium=referral)[Oct 18, 2025 11:02pm ET TRB Advisors LP Reduces Stake in Apple Inc by TipRanks](https://www.tipranks

: 

: 

: 

: 